In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
text = open("train.txt", "r", encoding="utf8").read()

In [3]:
def process_text(text):
    data = []
    items = text.split('\n')
    for item in items:
        d = item.split()
        try:
            if d[0]!= None:
                data.append(d)
        except:
            pass
    return data

In [4]:
data = process_text(text)

In [5]:
df = pd.DataFrame(data)

In [6]:
df

,0,1
0,Virulent,B-Malware
1,Android,B-OS
2,malware,O
3,returns,O
4,",",O
...,...,...
99901,to,O
99902,execute,O
99903,malicious,O
99904,macros,O


In [7]:
from ydata_profiling import ProfileReport

In [8]:
profile = ProfileReport(df)

In [9]:
profile.to_file("output.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 375.40it/s]


In [10]:
df = df.loc[df['0']!='the']
df = df.loc[df['0']!=',']
df = df.loc[df['0']!='.']
df = df.loc[df['0']!='to']
df = df.loc[df['0']!='and']
df = df.loc[df['0']!='of']
df = df.loc[df['0']!='a']
df = df.loc[df['0']!='in']
df = df.loc[df['0']!='that']
df = df.loc[df['0']!='is']
df = df.loc[df['0']!='<']
df = df.loc[df['0']!='>']
df = df.loc[df['0']!='=']

df


,0,1
0,Virulent,B-Malware
1,Android,B-OS
2,malware,O
3,returns,O
5,gets,O
...,...,...
99899,ActiveX,O
99900,control,O
99902,execute,O
99903,malicious,O


In [11]:
profile = ProfileReport(df)
profile.to_file("output2.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 322.84it/s]


In [12]:
def tokenize_letters(word):
    codes = []
    for char in word:
        codes.append(ord(char))
    return codes

In [13]:
max_len = 20
def pad_tokens(tokens):
    if len(tokens) < max_len:
        return tokens + [0] * (max_len - len(tokens))
    elif len(tokens) > max_len:
        return tokens[:max_len]
    else:
        return tokens

In [14]:
# df['tokens'] = df['0'].apply(tokenize_letters)
# max_len = 50
# for i in range(max_len): 
#     if len(df.loc[i,'0'])<max_len:
#         df.loc[i,'tokens'] = df.loc[i,'tokens']+[0]*(max_len-len(df.loc[i,'tokens']))
    
df['tokens'] = df['0'].apply(tokenize_letters)


df['tokens'] = df['tokens'].apply(pad_tokens)

In [15]:
df

,0,1,tokens
0,Virulent,B-Malware,"[86, 105, 114, 117, 108, 101, 110, 116, 0, 0, ..."
1,Android,B-OS,"[65, 110, 100, 114, 111, 105, 100, 0, 0, 0, 0,..."
2,malware,O,"[109, 97, 108, 119, 97, 114, 101, 0, 0, 0, 0, ..."
3,returns,O,"[114, 101, 116, 117, 114, 110, 115, 0, 0, 0, 0..."
5,gets,O,"[103, 101, 116, 115, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...,...
99899,ActiveX,O,"[65, 99, 116, 105, 118, 101, 88, 0, 0, 0, 0, 0..."
99900,control,O,"[99, 111, 110, 116, 114, 111, 108, 0, 0, 0, 0,..."
99902,execute,O,"[101, 120, 101, 99, 117, 116, 101, 0, 0, 0, 0,..."
99903,malicious,O,"[109, 97, 108, 105, 99, 105, 111, 117, 115, 0,..."


In [16]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
    word_index = lang_tokenizer.word_index
    return tensor, word_index

In [17]:
df['1'] = df['1'].str.replace('B-','')
df['1'] = df['1'].str.replace('I-','')


In [18]:
df

,0,1,tokens
0,Virulent,Malware,"[86, 105, 114, 117, 108, 101, 110, 116, 0, 0, ..."
1,Android,OS,"[65, 110, 100, 114, 111, 105, 100, 0, 0, 0, 0,..."
2,malware,O,"[109, 97, 108, 119, 97, 114, 101, 0, 0, 0, 0, ..."
3,returns,O,"[114, 101, 116, 117, 114, 110, 115, 0, 0, 0, 0..."
5,gets,O,"[103, 101, 116, 115, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...,...
99899,ActiveX,O,"[65, 99, 116, 105, 118, 101, 88, 0, 0, 0, 0, 0..."
99900,control,O,"[99, 111, 110, 116, 114, 111, 108, 0, 0, 0, 0,..."
99902,execute,O,"[101, 120, 101, 99, 117, 116, 101, 0, 0, 0, 0,..."
99903,malicious,O,"[109, 97, 108, 105, 99, 105, 111, 117, 115, 0,..."


In [19]:
y , y_index = tokenize(df['1'])

In [20]:
y

array([[3],
       [6],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [21]:
y = y[:, 0]
Y = pd.DataFrame(y)

In [22]:
y_index

{'o': 1,
 'application': 2,
 'malware': 3,
 'organization': 4,
 'malwaretype': 5,
 'os': 6,
 'location': 7,
 'time': 8,
 'threatactor': 9,
 'person': 10}

In [23]:
text2 = open("test.txt", "r", encoding="utf8").read()
test_data = process_text(text2)
df_test = pd.DataFrame(test_data)

In [24]:
df_test

,0,1
0,Ginp,B-Malware
1,-,O
2,A,O
3,malware,O
4,patchwork,O
...,...,...
20882,malware,O
20883,should,O
20884,be,O
20885,underestimated,O


In [25]:
df_test['tokens'] = df_test[0].apply(tokenize_letters)
df_test['tokens'] = df_test['tokens'].apply(pad_tokens)

In [26]:
df_test[1] = df_test[1].str.replace('B-','')
df_test[1] = df_test[1].str.replace('I-','')

In [27]:
y_test , y_test_index = tokenize(df_test[1])

In [28]:
y_test_index

{'o': 1,
 'application': 2,
 'malware': 3,
 'malwaretype': 4,
 'organization': 5,
 'os': 6,
 'time': 7,
 'location': 8,
 'threatactor': 9,
 'person': 10}

In [29]:
x_text = df_test['tokens']

In [30]:
X = df_test['tokens']

In [31]:
X

0        [71, 105, 110, 112, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
1        [45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
2        [65, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
3        [109, 97, 108, 119, 97, 114, 101, 0, 0, 0, 0, ...
4        [112, 97, 116, 99, 104, 119, 111, 114, 107, 0,...
                               ...                        
20882    [109, 97, 108, 119, 97, 114, 101, 0, 0, 0, 0, ...
20883    [115, 104, 111, 117, 108, 100, 0, 0, 0, 0, 0, ...
20884    [98, 101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
20885    [117, 110, 100, 101, 114, 101, 115, 116, 105, ...
20886    [46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
Name: tokens, Length: 20887, dtype: object

In [32]:
X = np.array(df['tokens'].tolist())
Y = np.array(df['1'].tolist())

In [33]:
X

array([[ 86, 105, 114, ...,   0,   0,   0],
       [ 65, 110, 100, ...,   0,   0,   0],
       [109,  97, 108, ...,   0,   0,   0],
       ...,
       [101, 120, 101, ...,   0,   0,   0],
       [109,  97, 108, ...,   0,   0,   0],
       [109,  97,  99, ...,   0,   0,   0]])

In [34]:
X = pd.DataFrame(X).values
Y = pd.DataFrame(Y).values

In [35]:
x_text = np.array(df_test['tokens'].tolist())
y_test = np.array(df_test[1].tolist())

In [36]:
x_text = pd.DataFrame(x_text).values
y_test = pd.DataFrame(y_test).values

In [37]:
x_text

array([[ 71, 105, 110, ...,   0,   0,   0],
       [ 45,   0,   0, ...,   0,   0,   0],
       [ 65,   0,   0, ...,   0,   0,   0],
       ...,
       [ 98, 101,   0, ...,   0,   0,   0],
       [117, 110, 100, ...,   0,   0,   0],
       [ 46,   0,   0, ...,   0,   0,   0]])

In [38]:
dt_model = DecisionTreeClassifier()
dt_model.fit(X,Y)

DecisionTreeClassifier()

In [39]:
y_pred = dt_model.predict(x_text)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

confusion_mtx = confusion_matrix(y_test, y_pred)
print('confusion_matrix')
print(confusion_mtx)

Accuracy: 0.944271556470532
Precision: 0.5862490808321772
Recall: 0.5463116685852778
F1-score: 0.5463064015217264
confusion_matrix
[[  294    23    14    24   181     3    11     4     0     0]
 [    4    59     1     1    15     0     2     5     5     0]
 [    5    91    96    25   128     1     9     6     1     0]
 [    0     0     0   175    64     2     0     0     0     0]
 [   72    18    10    27 18748    18    36    18    20    18]
 [    0     0     0     0    16   158     0     0     0     0]
 [   26    10    16     0    86     0    94     1     0     0]
 [    7     1     4     1    25     0     0     9     1     0]
 [    1     1     0     0    71     0     3    10     1     0]
 [    0     0     0     0    22     0     0     0     0    89]]


#علت تفاوت ارزیابی ها به این دلیل که ما توی دسته های به غیر از نگاتیو دقت مناسبی نداشتیم

In [40]:
dt_model = DecisionTreeClassifier(criterion='gini',splitter='best')
dt_model.fit(X,Y)

DecisionTreeClassifier()

In [41]:
y_pred = dt_model.predict(x_text)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
confusion_mtx = confusion_matrix(y_test, y_pred)
print('confusion_matrix')
print(confusion_mtx)

Accuracy: 0.9447024464978216
Precision: 0.6364482500174544
Recall: 0.546570724913342
F1-score: 0.5691811907351871
confusion_matrix
[[  307    17    12    24   180     3    11     0     0     0]
 [    4    53     1     7    18     0     7     2     0     0]
 [    3     2    97    25   218     1     9     6     1     0]
 [    0     0     0   175    64     2     0     0     0     0]
 [   85    20    11    26 18746    19    33    19     8    18]
 [    0     0     0     0    16   158     0     0     0     0]
 [   27    10    16     0    84     0    93     2     1     0]
 [   10     0     0     1    19     0     9     9     0     0]
 [    1     1     0     0    71     0     3     6     5     0]
 [    0     0     0     0    22     0     0     0     0    89]]


In [42]:
dt_model = DecisionTreeClassifier(criterion='entropy',splitter='best')
dt_model.fit(X,Y)

DecisionTreeClassifier(criterion='entropy')

In [43]:
y_pred = dt_model.predict(x_text)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
confusion_mtx = confusion_matrix(y_test, y_pred)
print('confusion_matrix')
print(confusion_mtx)

Accuracy: 0.9449897065160148
Precision: 0.6592478509848678
Recall: 0.5432930713794818
F1-score: 0.5702422965860711
confusion_matrix
[[  290    11     6    20   202     4    12     3     0     6]
 [    3    56     0     7    17     0     5     4     0     0]
 [    4     2    95    28   226     0     0     6     1     0]
 [    1     0     0   175    63     2     0     0     0     0]
 [   79    39    15    21 18766     3    26    11     7    18]
 [    0     0     0     0    17   157     0     0     0     0]
 [   21     7     8     5    94     0    96     0     2     0]
 [    8     9     1     0    24     0     1     5     0     0]
 [    3     7     0     0    65     0     2     1     9     0]
 [    0     0     0     0    22     0     0     0     0    89]]


In [44]:
dt_model = DecisionTreeClassifier(criterion='log_loss',splitter='best')
dt_model.fit(X,Y)

DecisionTreeClassifier(criterion='log_loss')

In [45]:
y_pred = dt_model.predict(x_text)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
confusion_mtx = confusion_matrix(y_test, y_pred)
print('confusion_matrix')
print(confusion_mtx)

Accuracy: 0.9447503231675205
Precision: 0.6187837384845603
Recall: 0.537454609996826
F1-score: 0.5501296887605449
confusion_matrix
[[  285     9     6    20   203     4    12     9     0     6]
 [    3    55     0     7    16     0     7     4     0     0]
 [   15     1    94   116   129     0     0     6     1     0]
 [    0     1     0   175    63     2     0     0     0     0]
 [   84    30    13    19 18770     5    26    12     6    20]
 [    0     0     0     0    16   158     0     0     0     0]
 [   21     7     8     5    91     0    97     0     4     0]
 [    7     9     1     0    25     0     1     5     0     0]
 [    3     7     1     0    65     0     6     0     5     0]
 [    0     0     0     0    20     2     0     0     0    89]]


In [46]:
for i in range(20):    
    rf_model = RandomForestClassifier(max_depth=i+1)
    dt_model.fit(X,Y)
    y_pred = dt_model.predict(x_text)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    print('********************')
    print(i+1)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

********************
1
Accuracy: 0.9447981998372194
Precision: 0.6500687613722432
Recall: 0.5506328349361643
F1-score: 0.5657493894196349
********************
2
Accuracy: 0.945229089864509
Precision: 0.6607508516717049
Recall: 0.54605504529382
F1-score: 0.5719813234143247
********************
3
Accuracy: 0.9449418298463159
Precision: 0.6606588903040713
Recall: 0.551701803980021
F1-score: 0.5774003352810082
********************
4
Accuracy: 0.9454205965433045
Precision: 0.661034820817593
Recall: 0.5507177866835548
F1-score: 0.5753132220010548
********************
5
Accuracy: 0.9448939531766171
Precision: 0.6612247601372035
Recall: 0.5514609434226007
F1-score: 0.5761683389166701
********************
6
Accuracy: 0.9448939531766171
Precision: 0.6646357195942248
Recall: 0.5519241925765029
F1-score: 0.5769588489000883
********************
7
Accuracy: 0.9445588164887251
Precision: 0.6572734394901335
Recall: 0.5563599622488206
F1-score: 0.5750988487031906
********************
8
Accuracy: 0.9452